ここに追加していく

In [1]:
!pip install requests beautifulsoup4


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [6]:
import requests
from bs4 import BeautifulSoup
import time
from urllib.parse import urljoin, urlparse
import json

In [12]:
import requests
from bs4 import BeautifulSoup
import time
from urllib.parse import urljoin, urlparse
import json

START_URL = "https://www.musashino-u.ac.jp/"

# 大学のドメインを取得
BASE_DOMAIN = urlparse(START_URL).netloc 

# 'www.' を除いた「ルート」ドメインを取得 (例: 'example.ac.jp')
if BASE_DOMAIN.startswith('www.'):
    ROOT_DOMAIN = BASE_DOMAIN.replace('www.', '', 1)
else:
    ROOT_DOMAIN = BASE_DOMAIN

# key: URL, value: <title>内の文字列
page_titles = {}

# これから訪問すべきURLのリスト
urls_to_visit = [START_URL]

# 重複と無限ループを防ぐため既に訪問したURLのセット
visited_urls = set()
visited_urls.add(START_URL)

print(f"--- クローリング開始: {START_URL} ---")

# urls_to_visit リストが空になるまでループ
#（※処理が多すぎると時間がかかりすぎるため、100ページで一旦停止する制限をかけます）
limit = 100 

while urls_to_visit and len(page_titles) < limit:
    # 1. リストの先頭からURLを1つ取り出す
    current_url = urls_to_visit.pop(0)

    print(f"[{len(page_titles)+1} / {limit}] Processing: {current_url}")

    # 2. 負荷軽減のためにスリープ
    time.sleep(1)

    try:
        # 3. Webページにアクセス
        response = requests.get(current_url, timeout=5)
        
        # 4. HTMLの文字コードをヘッダーから判定
        response.encoding = response.apparent_encoding
        
        # エラーならスキップ
        if response.status_code != 200:
            print(f"  -> Error: Status Code {response.status_code}")
            continue

        # 5. HTMLを解析
        soup = BeautifulSoup(response.text, 'html.parser')

        # 6. <title>を取得し、辞書に格納
        title = "No Title Found" 
        if soup.title and soup.title.string:
            title = soup.title.string.strip() # .strip()で前後の空白を削除
        
        page_titles[current_url] = title

        # 7. ページ内の全<a>タグ（リンク）を取得
        for link in soup.find_all('a'):
            href = link.get('href')

            # 8. リンクを絶対URLに変換
            if not href:
                continue # href属性がないタグは無視
            
            absolute_url = urljoin(current_url, href)
            
            # URLの#以降（ページ内リンク）や?以降（クエリ）を削除
            absolute_url = absolute_url.split('#')[0]
            absolute_url = absolute_url.split('?')[0]
            
            # .pdfや.jpgなどのファイルへのリンクは無視
            if any(ext in absolute_url.lower() for ext in ['.pdf', '.jpg', '.png', '.zip', '.xlsx', '.doc', '.ppt']):
                continue

            # 9. リンクをチェック
            link_netloc = urlparse(absolute_url).netloc
            
            # (a) 同じ大学のドメインか？
            is_same_university = link_netloc.endswith(ROOT_DOMAIN)

            # (b) まだ訪問済みリストに入っていないか？
            is_not_visited = (absolute_url not in visited_urls)

            if is_same_university and is_not_visited:
                urls_to_visit.append(absolute_url) # 訪問リストに追加
                visited_urls.add(absolute_url)     # 訪問済みリストに追加

    except requests.exceptions.RequestException as e:
        print(f"  -> Error accessing {current_url}: {e}")
        continue # エラーが起きても処理を続ける

# 10. 最後に結果をprint
print("\n--- クローリング完了 ---")
if len(page_titles) >= limit:
    print(f"（{limit}ページの上限に達しました）")

print("【取得結果（辞書型変数）】")
#print(page_titles)

print(json.dumps(page_titles, indent=2, ensure_ascii=False))

--- クローリング開始: https://www.musashino-u.ac.jp/ ---
[1 / 100] Processing: https://www.musashino-u.ac.jp/
[2 / 100] Processing: https://ef.musashino-u.ac.jp/donation/
[3 / 100] Processing: https://www.musashino-u.ac.jp/access.html
[4 / 100] Processing: https://www.musashino-u.ac.jp/admission/request.html
[5 / 100] Processing: https://www.musashino-u.ac.jp/contact.html
[6 / 100] Processing: https://www.musashino-u.ac.jp/prospective-students.html
[7 / 100] Processing: https://www.musashino-u.ac.jp/students.html
[8 / 100] Processing: https://www.musashino-u.ac.jp/alumni.html
[9 / 100] Processing: https://www.musashino-u.ac.jp/parents.html
[10 / 100] Processing: https://www.musashino-u.ac.jp/business.html
[11 / 100] Processing: https://www.musashino-u.ac.jp/guide/
[12 / 100] Processing: https://www.musashino-u.ac.jp/guide/profile/
[13 / 100] Processing: https://www.musashino-u.ac.jp/guide/activities/
[14 / 100] Processing: https://www.musashino-u.ac.jp/guide/campus/
[15 / 100] Processing: http